In [1]:
import random
import pathlib

from typing import Optional

import click
from src.schnapsen.bots.rdeep_ML import RdeepMLBot
from src.schnapsen.bots import MLDataBot, train_ML_model, MLPlayingBot, RandBot, BullyBot
from src.schnapsen.bots.example_bot import ExampleBot

from src.schnapsen.game import (Bot, Move, PlayerPerspective,
                            SchnapsenGamePlayEngine, Trump_Exchange)
from src.schnapsen.twenty_four_card_schnapsen import \
    TwentyFourSchnapsenGamePlayEngine
from src.schnapsen.bots.bot2 import SecondBot
from src.schnapsen.bots.rdeep import RdeepBot
from binomial_experiment import binom_experiment

def create_replay_memory_dataset(bot1, bot2, filename) -> None:
    
    # define replay memory database creation parameters
    num_of_games: int = 30000
    replay_memory_dir: str = 'ML_replay_memories'
    replay_memory_filename: str = filename
    replay_memory_location = pathlib.Path(replay_memory_dir) / replay_memory_filename
    delete_existing_older_dataset = False

    # check if needed to delete any older versions of the dataset
    if delete_existing_older_dataset and replay_memory_location.exists():
        print(f"An existing dataset was found at location '{replay_memory_location}', which will be deleted as selected.")
        replay_memory_location.unlink()

    # in any case make sure the directory exists
    replay_memory_location.parent.mkdir(parents=True, exist_ok=True)

    # create new replay memory dataset, according to the behaviour of the provided bots and the provided random seed
    engine = SchnapsenGamePlayEngine()
    replay_memory_recording_bot_1 = MLDataBot(bot1, replay_memory_location=replay_memory_location)
    replay_memory_recording_bot_2 = MLDataBot(bot2, replay_memory_location=replay_memory_location)
    for i in range(1, num_of_games + 1):
        if i % 500 == 0:
            print(f"Progress: {i}/{num_of_games}")
        engine.play_game(replay_memory_recording_bot_1, replay_memory_recording_bot_2, random.Random(i))
    print(f"Replay memory dataset recorder for {num_of_games} games.\nDataset is stored at: {replay_memory_location}")


In [ ]:
create_replay_memory_dataset(RandBot(random.Random(5234243)), RandBot(random.Random(54354)), "random_random_30k_games_for_rollout.txt")

In [ ]:
create_replay_memory_dataset(BullyBot(random.Random(5234243)), BullyBot(random.Random(54354)), "bully_bully_30k_games_for_rollout.txt")

In [ ]:
create_replay_memory_dataset(RdeepBot(6,6,random.Random(5234243)), RdeepBot(6,6,random.Random(54354)), "rdeep_rdeep_30k_games_for_rollout.txt")

In [ ]:
create_replay_memory_dataset(SecondBot(random.Random(5234243)), SecondBot(random.Random(54354)), "2ndBot_2ndBot_30k_games_for_rollout.txt")

In [1]:
import pandas as pd
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
replay_memory_location_bully = "ML_replay_memories/bully_bully_30k_games_for_rollout.txt"
replay_memory_location_rand = "ML_replay_memories/random_random_30k_games_for_rollout.txt"
replay_memory_location_rdeep = "ML_replay_memories/rdeep_rdeep_30k_games_for_rollout.txt"
replay_memory_location_2nd_bot = "ML_replay_memories/2ndBot_2ndBot_30k_games_for_rollout.txt"

In [3]:
# labbels
# 0 rand_bot
# 1 bully_bot
# 2 rdeep_bot
# 3 2nd_bot

In [4]:
data_rand: list[list[int]] = []
targets_rand: list[int] = []
with open(file=replay_memory_location_rand, mode="r") as replay_memory_file:
    for line in replay_memory_file:
        feature_string, won_label_str = line.split("||")
        feature_list_strings: list[str] = feature_string.split(",")
        feature_list = [float(feature) for feature in feature_list_strings]
        won_label = int(won_label_str)
        data_rand.append(feature_list)
        targets_rand.append(0)

In [5]:
data_bully: list[list[int]] = []
targets_bully: list[int] = []
with open(file=replay_memory_location_bully, mode="r") as replay_memory_file:
    for line in replay_memory_file:
        feature_string, won_label_str = line.split("||")
        feature_list_strings: list[str] = feature_string.split(",")
        feature_list = [float(feature) for feature in feature_list_strings]
        won_label = int(won_label_str)
        data_bully.append(feature_list)
        targets_bully.append(1)

In [6]:
data_rdeep: list[list[int]] = []
targets_rdeep: list[int] = []
with open(file=replay_memory_location_rdeep, mode="r") as replay_memory_file:
    for line in replay_memory_file:
        feature_string, won_label_str = line.split("||")
        feature_list_strings: list[str] = feature_string.split(",")
        feature_list = [float(feature) for feature in feature_list_strings]
        won_label = int(won_label_str)
        data_rdeep.append(feature_list)
        targets_rdeep.append(2)

In [7]:
data_2ndBot: list[list[int]] = []
targets_2ndBot: list[int] = []
with open(file=replay_memory_location_2nd_bot, mode="r") as replay_memory_file:
    for line in replay_memory_file:
        feature_string, won_label_str = line.split("||")
        feature_list_strings: list[str] = feature_string.split(",")
        feature_list = [float(feature) for feature in feature_list_strings]
        won_label = int(won_label_str)
        data_2ndBot.append(feature_list)
        targets_2ndBot.append(3)

In [8]:
data = data_rand + data_bully + data_rdeep + data_2ndBot
target = targets_rand + targets_bully + targets_rdeep + targets_2ndBot

In [9]:
train_data, test_data, train_targets, test_targets = train_test_split(data, target, test_size = 0.3, random_state=4545, shuffle = True)

In [14]:
model = KNeighborsClassifier(n_neighbors=5)


In [16]:
model.fit(train_data,train_targets)

KNeighborsClassifier()

In [18]:
model.score(test_data, test_targets)

0.5608373003118057

In [17]:
model_location = "ML_models/KNN_model"
joblib.dump(model, model_location)

['ML_models/KNN_model']